<div style="text-align: center; margin-left: 0em; font-weight: bold; font-size: 20px; font-family: TimesNewRoman;">
    TIME SERIES DATA PROCESSING | NET TRANSFER CAPACIY - Downloading Notebook
</div>

<div style="text-align: left; margin-left: 0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Each part of the following script is used to donwload the raw data for the Cross Border Flows Time Series Raw Data for all the european countries of the Dispa-SET_Unleash project.
<br>
Read explanation text cells to follow and understand all the process until final results were got stept by step.
</div>
<br>
<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    1. Notebook Set Up
</div>

<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Importing needed libraries
</div>

In [65]:
import pandas as pd
from entsoe import EntsoePandasClient
import os
import csv
from datetime import datetime

 <div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    2. ENTSO-E RESTful API.
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Connecting with the ENTSO-E API Tool.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- To donwload the neeeded data using the API tool, is mandatory to use a token autentication to connect and make the future request of data.
</div>

In [2]:
client = EntsoePandasClient(api_key='61e5bbbb-7e80-4540-a471-bd993873aa74')

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    3. Dispa-SET_Unleash Folder Path
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
    Determinning dynamically the zone_folder_path based on the location of the "Dispa-SET_Unleash" folder relative to the current working directory.
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- If the "Dispa-SET_Unleash" folder is copied to a different machine or location, the dispaSET_unleash_folder_path variable will automatically adjust accordingly.
</div>

In [3]:
# Get the current working directory
current_directory = os.getcwd()

# Navigate to the parent directory of "Dispa-SET_Unleash"
dispaSET_unleash_parent_directory = os.path.dirname(current_directory)

# Get the path to the "Dispa-SET_Unleash" folder
dispaSET_unleash_folder_path = os.path.dirname(dispaSET_unleash_parent_directory)

# Construct the dispaSET_unleash_folder_name variable
dispaSET_unleash_folder_name = os.path.basename(dispaSET_unleash_folder_path)

print("dispaSET_unleash_folder_name:", dispaSET_unleash_folder_name)
print("dispaSET_unleash_folder_path:", dispaSET_unleash_folder_path)

dispaSET_unleash_folder_name: Dispa-SET_Unleash
dispaSET_unleash_folder_path: /home/ray/Dispa-SET_Unleash


<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    4. Usefull Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Entering a value to all the variables which content are going to be used in some of the next stages of this script. 
</div>
<div style="text-align: left; margin-left: 2.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
- Indicate the year of all data is referring to in the variable data_year.
<br>
- The universal_standar_time variable is going to be used to download all the time series data in this horary zone. Additionally as each european country belongs a particular time sector the corresponding time series data related to its time sector are going to be downloaded as well but in a different file.
<br>
- Additionally there are some default parameters that has to be defined to the correct working and calling to the ENTSO-E downloading functions.
</div>

In [4]:
# Year to which data refers to:
data_year = 2023

In [57]:
# Additional string to be appended
additional_path = "/RawData/NTC/"
additional_path_1 = "/RawData/NTC/Raw_Data_Sources/"

# Construct the Outage_Factors_folder_path variable
ntc_folder_path = dispaSET_unleash_folder_path + additional_path

# Construct the Outage_Factors_Raw_Data_folder_path variable
ntc_raw_data_folder_path = dispaSET_unleash_folder_path + additional_path_1

print("ntc_folder_path:", ntc_folder_path)
print("ntc_raw_data_folder_path:", ntc_raw_data_folder_path)

ntc_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/
ntc_raw_data_folder_path: /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/


In [29]:
# Define other parameters
type_marketagreement_type = 'A01'
contract_marketagreement_type = "A01"
process_type = 'A51'

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    5. Country List Variable Definition
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the list of countries according to the available data. 
</div>

In [58]:
# Define a list of country codes
ntc_per_unit_country_list = ["AT", "BE", "BG", "CH", "CY", "CZ", "DE", "DK", "EE", "GR", "ES", "FI", "FR", "HR", "HU", 
                             "IE", "IT", "LT", "LU", "LV", "MT", "NL", "NO", "PL", "PT", "RO", "SE", "SI", "SK", "UK"]

In [59]:
# Define the directory and file path
file_name = 'country_list.csv'
file_path = os.path.join(ntc_raw_data_folder_path, file_name)

# Ensure the directory exists
os.makedirs(ntc_raw_data_folder_path, exist_ok=True)

# Create a DataFrame
df = pd.DataFrame(ntc_per_unit_country_list, columns=['Country_From'])

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

print(f"DataFrame saved to '{file_path}'")
ntc_country_list_file = file_path

DataFrame saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv'


In [60]:
print (f"dispaSET_unleash_folder_name:                              {dispaSET_unleash_folder_name}")
print (f"dispaSET_unleash_folder_path:                              {dispaSET_unleash_folder_path}")
print (f"data_year:                                                 {data_year}")
print (f"ntc_raw_data_folder_path:                                  {ntc_raw_data_folder_path}")
print (f"ntc_country_list_file:                                     {ntc_country_list_file}")

dispaSET_unleash_folder_name:                              Dispa-SET_Unleash
dispaSET_unleash_folder_path:                              /home/ray/Dispa-SET_Unleash
data_year:                                                 2023
ntc_raw_data_folder_path:                                  /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/
ntc_country_list_file:                                     /home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the sub-folders of all the net transfer capacity raw data is saved. 
</div>

In [63]:
# Convert data_year to string if it's not already
data_year = str(data_year)

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the column 'Country_From' exists
if 'Country_From' not in df.columns:
    raise ValueError("Column 'Country_From' does not exist in the CSV file")

# Define the base directory where subfolders will be created
base_directory = os.path.join(ntc_raw_data_folder_path, data_year)

# Create a list to hold the paths of the created subfolders
country_folder_paths = []

# Create subfolders and save their paths
for country in df['Country_From']:
    # Create the subfolder path
    subfolder_path = os.path.join(base_directory, country)
    
    # Create the subfolder if it doesn't exist
    os.makedirs(subfolder_path, exist_ok=True)
    
    # Append the subfolder path to the list
    country_folder_paths.append(subfolder_path)

# Add the new column 'Country_Folder' to the DataFrame
df['Country_Folder'] = country_folder_paths

# Save the updated DataFrame back to the CSV file
df.to_csv(ntc_country_list_file, index=False)

print(f"Updated CSV file saved to '{ntc_country_list_file}' with new subfolders created.")

Updated CSV file saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv' with new subfolders created.


<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Defining the neighbor countries. 
</div>

In [64]:
# Raw data as a multiline string
data = """
CZ, DE, HU, IT, SI, CH
FR, DE, LU, NL, UK
GR, RO, 
AT, FR, DE, IT

AT, DE, PL, SK
AT, BE, CZ, DK, FR, LU, NL, NO, PL, SE, CH
DE, NL, NO, SE, UK
FI, LV
BG, IT
FR, PT
EE, NO, SE
BE, DE, IT, ES, CH, UK
HU, SI
AT, HR, RO, SK, SI
UK
AT, FR, GR, MT, SI, CH
LV, PL, SE
BE, DE
EE, LT
IT
BE, DK, DE, NO, UK
DK, FI, DE, NL, SE, UK
CZ, DE, LT, SK, SE
ES
BG, HU
DK, FI, DE, LT, NO, PL
AT, HR, HU, IT
CZ, HU, PL
BE, DK, FR, IE, NL, NO
"""

# Split the data into lines
lines = data.strip().split("\n")

# Initialize a list to hold the data
data_list = []

# Process each line
for line in lines:
    if line.strip() == "":
        # If the line is empty, add 11 empty strings
        data_list.append([""] * 11)
    else:
        neighbors = line.split(", ")
        data_list.append(neighbors)

# Create the DataFrame for new data
new_df = pd.DataFrame(data_list, columns=[f"Neighbor_{i}" for i in range(1, 12)])

# Path to the existing CSV file
existing_csv_file_path = ntc_country_list_file

# Read the existing CSV file into a DataFrame
existing_df = pd.read_csv(existing_csv_file_path)

# Concatenate the existing DataFrame with the new DataFrame horizontally
combined_df = pd.concat([existing_df, new_df], axis=1)

# Save the combined DataFrame back to the CSV file
combined_df.to_csv(existing_csv_file_path, index=False)
print(f"Data appended to '{existing_csv_file_path}' as new columns")

Data appended to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv' as new columns


In [66]:
# Define the start and end timestamps
start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = pd.Timestamp('20240101', tz='Europe/Brussels')

# Path to the CSV file
#ntc_country_list_file = '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder'] + [f'Neighbor_{i}' for i in range(1, 12)]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_code_from = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Iterate through each neighbor column
    for neighbor_col in [f'Neighbor_{i}' for i in range(1, 12)]:
        country_code_to = row[neighbor_col]
        
        # Skip if the neighbor field is empty
        if pd.isna(country_code_to):
            continue
        
        # Query crossborder flows
        net_transfer = client.query_crossborder_flows(
            country_code_from, country_code_to, start=start, end=end
        )
        
        # Define the output file path
        output_file = os.path.join(country_folder, f'{country_code_to}.csv')
        
        # Save the DataFrame to a CSV file
        net_transfer.to_csv(output_file, index=False)

        print(f"Data for {country_code_from} to {country_code_to} saved to '{output_file}'")

print("All data has been processed and saved.")


Data for AT to CZ saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CZ.csv'
Data for AT to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/DE.csv'
Data for AT to HU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/HU.csv'
Data for AT to IT saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/IT.csv'
Data for AT to SI saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/SI.csv'
Data for AT to CH saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CH.csv'
Data for BE to FR saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/FR.csv'
Data for BE to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/DE.csv'
Data for BE to LU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/LU.csv'
Data for BE to NL saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/B

In [68]:
# Define the start and end timestamps
start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = pd.Timestamp('20240101', tz='Europe/Brussels')

# Path to the CSV file
#ntc_country_list_file = '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/country_list.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(ntc_country_list_file)

# Ensure the necessary columns exist
required_columns = ['Country_From', 'Country_Folder'] + [f'Neighbor_{i}' for i in range(1, 12)]
for col in required_columns:
    if col not in df.columns:
        raise ValueError(f"Column '{col}' does not exist in the CSV file")

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    country_code_from = row['Country_From']
    country_folder = row['Country_Folder']
    
    # Create the directory if it does not exist
    os.makedirs(country_folder, exist_ok=True)
    
    # Iterate through each neighbor column
    for neighbor_col in [f'Neighbor_{i}' for i in range(1, 12)]:
        country_code_to = row[neighbor_col]
        
        # Skip if the neighbor field is empty
        if pd.isna(country_code_to):
            continue
        
        # Query crossborder flows
        net_transfer = client.query_crossborder_flows(
            country_code_from, country_code_to, start=start, end=end
        )
        
        # Convert the index to a column
        net_transfer = net_transfer.reset_index()
        
        # Define the output file path
        output_file = os.path.join(country_folder, f'{country_code_to}.csv')
        
        # Save the DataFrame to a CSV file, including the index as a column
        net_transfer.to_csv(output_file, index=False)

        print(f"Data for {country_code_from} to {country_code_to} saved to '{output_file}'")

print("All data has been processed and saved.")


Data for AT to CZ saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CZ.csv'
Data for AT to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/DE.csv'
Data for AT to HU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/HU.csv'
Data for AT to IT saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/IT.csv'
Data for AT to SI saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/SI.csv'
Data for AT to CH saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/AT/CH.csv'
Data for BE to FR saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/FR.csv'
Data for BE to DE saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/DE.csv'
Data for BE to LU saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/BE/LU.csv'
Data for BE to NL saved to '/home/ray/Dispa-SET_Unleash/RawData/NTC/Raw_Data_Sources/2023/B

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    6. Outages Data Per Unit Downloading
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Downloading the outages raw data which countries have a desagregation per unit. 
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    8. Non Outages Data
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Creating empty csv files with the corresponding headers for those countries which outages raw data are not available.
</div>

<div style="text-align: left; margin-left: 3.0em; font-weight: bold; font-size: 18px; font-family: TimesNewRoman;">
    9. Greece Name
</div>
<div style="text-align: left; margin-left: 0.0em; font-weight: unbold; font-size: 16px; font-family: TimesNewRoman;">
Changing the Acronym of Greece country to the correspondign to the Dispa-SET nomenclature.
</div>